In [1]:
#!pip install beautifulsoup4
#!pip install requests
#!pip install tqdm
#!pip install pandas

import bs4 as bs # BeautifulSoup para analizar el HTML
import requests # Para hacer peticiones HTTP a páginas web
from tqdm import tqdm # Para mostrar barra de progreso del código
import pandas as pd 

In [2]:
# Departamentos del Perú 
departamentos = [
    "amazonas", "ancash", "apurimac", "arequipa", "ayacucho", "cajamarca",
    "cusco", "huancavelica", "huanuco", "ica", "junin", "la-libertad",
    "lambayeque", "lima", "loreto", "madre-de-dios", "moquegua", "pasco",
    "piura", "puno", "san-martin", "tacna", "tumbes", "ucayali"
]

Fechas = []
Lugar = []
Festividades = []
Departamento = []



for dpto in tqdm(departamentos):
    try:
        url = f'https://www.serperuano.com/turismo/turismo-en-{dpto}/calendario-festivo-en-{dpto}/'
        resp = requests.get(url)
        soup = bs.BeautifulSoup(resp.text, 'lxml')
        table = soup.find('table', {'class': 'stcelda02'})

        if table:
            for row in table.findAll('tr')[1:]:
                cols = row.findAll('td')
                if len(cols) >= 3:
                    Fechas.append(cols[0].text.strip())
                    Lugar.append(cols[1].text.strip())
                    Festividades.append(cols[2].text.strip())
                    Departamento.append(dpto.replace("-", " ").title())

        else:
            print(f" {dpto.title()}: No se encontró tabla")

    except Exception as e:
        print(f" {dpto.title()}: Error → {e}")

# Crear DataFrame
df = pd.DataFrame(list(zip(Departamento, Fechas, Lugar, Festividades)),
                  columns=['Departamento', 'Fechas', 'Lugar', 'Festividades'])

df = df.reset_index(drop=True)

# Mostrar los primeros registros
print(f"\n Total de festividades extraídas: {len(df)}")
df.head()


  0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 24/24 [00:12<00:00,  1.88it/s]


 Total de festividades extraídas: 777


,Departamento,Fechas,Lugar,Festividades
0,Amazonas,Febrero (Movible),Todo el departamento,Carnavales.
1,Amazonas,1-6 de mayo,Todo el departamento,Fiesta de las Cruces.
2,Amazonas,24 de junio,Bongará,Fiesta de San Juan.
3,Amazonas,25-29 de junio,Todo el departamento,Festividad de San Pedro y San Pablo.
4,Amazonas,Movible,Chachapoyas,Semana Turística.


In [3]:
df.to_csv("festividades_departamentos_peru.csv", index=False, encoding="utf-8-sig")
